### Библиотекаларды қосу

In [3]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import pickle

In [4]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
if currentdir+'/lib/' not in sys.path:
    sys.path.append(currentdir+'/lib/')

import requestsparsingVk

path_models = 'models/'

### Функцияларды қосу

In [5]:
def number_to_class(arr):
    result = ''
    for i in arr:
        result += class_names[i] + ' '
    return result

In [6]:
import re

def cleaning_data(code_data):
    text = code_data
    text = text.replace('(<br/>)', '')
    text = text.replace('(<a).*(>).*(</a>)', '')
    text = text.replace('(&amp)', '')
    text = text.replace('(&gt)', '')
    text = text.replace('(&lt)', '')
    text = text.replace('(\xa0)', ' ')
    text = text.replace('-', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace(',', ' ')
    text = filtering(text)
    return text.strip()

def filtering(text):
    stripped = re.sub('[^a-zA-Z, ^А-Я,а-я,Ә,І,Ң,Ғ,Ү,Ұ,Қ,Ө,Һ,ә,і,ə,ң,ғ,ү,ұ,қ,ө,һ]', ' ', str(text).replace('-', ''))
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    return str(stripped).lower()

### Модельдерді қосу

In [7]:
with open(path_models+'vectorizer.pickle', 'rb') as handle:
    vectorizer = pickle.load(handle)

In [8]:
with open(path_models+'modellr.pickle', 'rb') as handle:
    model = pickle.load(handle)

In [9]:
with open(path_models+'class_names.pickle', 'rb') as handle:
    class_names = pickle.load(handle)

### Parsing Жасау

In [10]:
%%time
token = '8bbb4daf8bbb4daf8bbb4daf0c8bcf8d4a88bbb8bbb4dafebb491b8847b23c3736b59f0'
url = 'kaz_sport2016'
parse = requestsparsingVk.parsingVk(token)
data = pd.DataFrame(parse.get_posts(url))
data.columns = ['id', 'from_id', 'text']
data[class_names]=None

Wall time: 9min 52s


### Классификациялау

In [11]:
data['text'] = data['text'].apply(lambda x: cleaning_data(x))
data

,id,from_id,text,01_auto,02_economics_business,03_entertainment,04_family,05_fashion,06_games,07_health,08_politics,09_realty,10_science_hitech,11_sport,12_travel
0,273899,-126481525,футбол уефа чемпиондар лигасы финал порту порт...,None,None,None,None,None,None,None,None,None,None,None,None
1,273891,-126481525,алексей щ ткин ротор сапындағы кезекті голын с...,None,None,None,None,None,None,None,None,None,None,None,None
2,273887,-126481525,футболдан үздік әлемдік төреші анықталды халық...,None,None,None,None,None,None,None,None,None,None,None,None
3,273886,-126481525,алексей бага мен команданың агрессивті футбол ...,None,None,None,None,None,None,None,None,None,None,None,None
4,273885,-126481525,қазақстан құрамасы күзде грузиямен жолдастық к...,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73054,7,-126481525,қазақ дзюдосының кешегісі мен бүгіні асхат жіт...,None,None,None,None,None,None,None,None,None,None,None,None
73055,6,-126481525,бокстан құрамамыздың қай күндері жұдырықтасаты...,None,None,None,None,None,None,None,None,None,None,None,None
73056,5,-126481525,елдостың жеңісі дидар хамзаға жұғысты болсын,None,None,None,None,None,None,None,None,None,None,None,None
73057,2,-126481525,арли чонтейде елесееваның жолын қайталап жүлде...,None,None,None,None,None,None,None,None,None,None,None,None


In [17]:
data = data[data['text'].apply(lambda x: True if len(x.split(' ')) > 5 else False)]
data

,id,from_id,text,01_auto,02_economics_business,03_entertainment,04_family,05_fashion,06_games,07_health,08_politics,09_realty,10_science_hitech,11_sport,12_travel
0,273899,-126481525,футбол уефа чемпиондар лигасы финал порту порт...,None,None,None,None,None,None,None,None,None,None,None,None
1,273891,-126481525,алексей щ ткин ротор сапындағы кезекті голын с...,None,None,None,None,None,None,None,None,None,None,None,None
2,273887,-126481525,футболдан үздік әлемдік төреші анықталды халық...,None,None,None,None,None,None,None,None,None,None,None,None
3,273886,-126481525,алексей бага мен команданың агрессивті футбол ...,None,None,None,None,None,None,None,None,None,None,None,None
4,273885,-126481525,қазақстан құрамасы күзде грузиямен жолдастық к...,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73053,8,-126481525,мария дмитренко іріктеу кезеңінде ші орын алды...,None,None,None,None,None,None,None,None,None,None,None,None
73054,7,-126481525,қазақ дзюдосының кешегісі мен бүгіні асхат жіт...,None,None,None,None,None,None,None,None,None,None,None,None
73055,6,-126481525,бокстан құрамамыздың қай күндері жұдырықтасаты...,None,None,None,None,None,None,None,None,None,None,None,None
73056,5,-126481525,елдостың жеңісі дидар хамзаға жұғысты болсын,None,None,None,None,None,None,None,None,None,None,None,None


In [18]:
X = vectorizer.transform(data['text'])

In [19]:
data[class_names] = np.round(model.predict_proba(X), 2)
data

c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,id,from_id,text,01_auto,02_economics_business,03_entertainment,04_family,05_fashion,06_games,07_health,08_politics,09_realty,10_science_hitech,11_sport,12_travel
0,273899,-126481525,футбол уефа чемпиондар лигасы финал порту порт...,0.06,0.05,0.04,0.11,0.03,0.04,0.02,0.03,0.04,0.04,0.48,0.07
1,273891,-126481525,алексей щ ткин ротор сапындағы кезекті голын с...,0.02,0.01,0.03,0.02,0.01,0.02,0.01,0.03,0.02,0.01,0.81,0.01
2,273887,-126481525,футболдан үздік әлемдік төреші анықталды халық...,0.06,0.03,0.03,0.03,0.05,0.02,0.02,0.05,0.02,0.03,0.58,0.10
3,273886,-126481525,алексей бага мен команданың агрессивті футбол ...,0.03,0.03,0.02,0.04,0.01,0.05,0.01,0.03,0.01,0.03,0.72,0.02
4,273885,-126481525,қазақстан құрамасы күзде грузиямен жолдастық к...,0.03,0.04,0.08,0.02,0.02,0.02,0.01,0.21,0.04,0.01,0.49,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73053,8,-126481525,мария дмитренко іріктеу кезеңінде ші орын алды...,0.09,0.03,0.09,0.05,0.02,0.02,0.02,0.02,0.02,0.03,0.56,0.04
73054,7,-126481525,қазақ дзюдосының кешегісі мен бүгіні асхат жіт...,0.06,0.05,0.05,0.09,0.08,0.02,0.03,0.05,0.05,0.05,0.35,0.11
73055,6,-126481525,бокстан құрамамыздың қай күндері жұдырықтасаты...,0.15,0.04,0.06,0.25,0.04,0.05,0.02,0.04,0.04,0.05,0.10,0.16
73056,5,-126481525,елдостың жеңісі дидар хамзаға жұғысты болсын,0.13,0.08,0.05,0.26,0.05,0.05,0.02,0.04,0.06,0.06,0.09,0.11


In [20]:
data['three best classes'] = list(map(number_to_class, (-np.round(model.predict_proba(X), 2)).argsort(axis=1)[:, :3]))

c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
data

,id,from_id,text,01_auto,02_economics_business,03_entertainment,04_family,05_fashion,06_games,07_health,08_politics,09_realty,10_science_hitech,11_sport,12_travel,three best classes
0,273899,-126481525,футбол уефа чемпиондар лигасы финал порту порт...,0.06,0.05,0.04,0.11,0.03,0.04,0.02,0.03,0.04,0.04,0.48,0.07,11_sport 04_family 12_travel
1,273891,-126481525,алексей щ ткин ротор сапындағы кезекті голын с...,0.02,0.01,0.03,0.02,0.01,0.02,0.01,0.03,0.02,0.01,0.81,0.01,11_sport 03_entertainment 08_politics
2,273887,-126481525,футболдан үздік әлемдік төреші анықталды халық...,0.06,0.03,0.03,0.03,0.05,0.02,0.02,0.05,0.02,0.03,0.58,0.10,11_sport 12_travel 01_auto
3,273886,-126481525,алексей бага мен команданың агрессивті футбол ...,0.03,0.03,0.02,0.04,0.01,0.05,0.01,0.03,0.01,0.03,0.72,0.02,11_sport 06_games 04_family
4,273885,-126481525,қазақстан құрамасы күзде грузиямен жолдастық к...,0.03,0.04,0.08,0.02,0.02,0.02,0.01,0.21,0.04,0.01,0.49,0.04,11_sport 08_politics 03_entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73053,8,-126481525,мария дмитренко іріктеу кезеңінде ші орын алды...,0.09,0.03,0.09,0.05,0.02,0.02,0.02,0.02,0.02,0.03,0.56,0.04,11_sport 01_auto 03_entertainment
73054,7,-126481525,қазақ дзюдосының кешегісі мен бүгіні асхат жіт...,0.06,0.05,0.05,0.09,0.08,0.02,0.03,0.05,0.05,0.05,0.35,0.11,11_sport 12_travel 04_family
73055,6,-126481525,бокстан құрамамыздың қай күндері жұдырықтасаты...,0.15,0.04,0.06,0.25,0.04,0.05,0.02,0.04,0.04,0.05,0.10,0.16,04_family 12_travel 01_auto
73056,5,-126481525,елдостың жеңісі дидар хамзаға жұғысты болсын,0.13,0.08,0.05,0.26,0.05,0.05,0.02,0.04,0.06,0.06,0.09,0.11,04_family 01_auto 12_travel


In [22]:
data.to_csv('dataset/vk_results.csv', index=False)